# AutoGen Fundamentals: Conversable Agents & Tool Integration

This notebook introduces **AutoGen 0.4**, Microsoft's multi-agent framework. Key architectural concepts:
1. **Model Client**: Abstraction for LLM providers (OpenAI, Anthropic, Ollama)
2. **Messages**: Typed message objects for inter-agent communication
3. **Agents**: Autonomous entities with defined behaviors and tool access

**Pattern:** AutoGen enables conversational AI systems where agents can call tools, reflect on results, and collaborate to solve tasks.

In [ ]:
# Initialize Environment
from dotenv import load_dotenv
load_dotenv(override=True)

## Phase 1: Configure Model Clients

In [ ]:
# OpenAI Model Client
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [ ]:
# Alternative: Ollama for Local Inference
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollama_client = OllamaChatCompletionClient(model="llama3.2")

## Phase 2: Define Messages

In [ ]:
# Create Message Object
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

## Phase 3: Create Agent

In [ ]:
# Assistant Agent
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful airline assistant. Provide concise, witty responses.",
    model_client_stream=True
)

In [ ]:
# Execute Agent
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

## Phase 4: Add Tools (SQLite Database Integration)

In [ ]:
# Create Local Price Database
import os
import sqlite3

# Initialize Database
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [ ]:
# Populate Database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Sample Data
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [ ]:
# Define Tool Function
def get_city_price(city_name: str) -> float | None:
    """Get the roundtrip ticket price for a city"""
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

# Test
get_city_price("Rome")

## Phase 5: Tool-Enabled Agent

In [ ]:
# Agent with Tool Access
smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful airline assistant. Provide concise, witty responses with pricing information.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True  # Agent reflects on tool results before responding
)

In [ ]:
# Execute with Tool Usage
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())

# Show Internal Reasoning (Tool Calls)
for inner_message in response.inner_messages:
    print(f"[Internal] {inner_message.content}")

# Final Response
print(f"\n[Response] {response.chat_message.content}")